# Link Prediction

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dmlc/dgl/blob/master/notebooks/stochastic_training/link_prediction.ipynb) [![GitHub](https://img.shields.io/badge/-View%20on%20GitHub-181717?logo=github&logoColor=ffffff)](https://github.com/dmlc/dgl/blob/master/notebooks/stochastic_training/link_prediction.ipynb)

This tutorial will show how to train a multi-layer GraphSAGE for link
prediction on [CoraGraphDataset](https://data.dgl.ai/dataset/cora_v2.zip).
The dataset contains 2708 nodes and 10556 edges.

By the end of this tutorial, you will be able to

-  Train a GNN model for link prediction on target device with DGL's
   neighbor sampling components.


## Install DGL package

In [1]:
# Install required packages.
import os
import torch
import numpy as np
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

# Install the CPU version. If you want to install CUDA version, please
# refer to https://www.dgl.ai/pages/start.html.
device = torch.device("cpu")
!pip install --pre dgl -f https://data.dgl.ai/wheels-test/repo.html

try:
    import dgl
    import dgl.graphbolt as gb
    installed = True
except ImportError as error:
    installed = False
    print(error)
print("DGL installed!" if installed else "DGL not found!")

Looking in links: https://data.dgl.ai/wheels-test/repo.html


DGL installed!


## Loading Dataset
`cora` is already prepared as `BuiltinDataset` in **GraphBolt**.


In [2]:
dataset = gb.BuiltinDataset("cora").load()

datasets/cora.zip:   0%|                                              | 0.00/240k [00:00<?, ?B/s]

datasets/cora.zip: 100%|██████████████████████████████████████| 240k/240k [00:00<00:00, 7.28MB/s]

Extracting file to datasets


Start to preprocess the on-disk dataset.
Finish preprocessing the on-disk dataset.



/home/ubuntu/regression_test/dgl/python/dgl/graphbolt/impl/ondisk_dataset.py:467: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(graph_topology.path)


Dataset consists of graph, feature and tasks. You can get the training-validation-test set from the tasks. Seed nodes and corresponding labels are already stored in each training-validation-test set. This dataset contains 2 tasks, one for node classification and the other for link prediction. We will use the link prediction task.

In [3]:
graph = dataset.graph
feature = dataset.feature
train_set = dataset.tasks[1].train_set
test_set = dataset.tasks[1].test_set
task_name = dataset.tasks[1].metadata["name"]
print(f"Task: {task_name}.")

Task: link_prediction.


## Defining Neighbor Sampler and Data Loader in DGL
Different from the link prediction tutorial for full graph, a common practice to train GNN on large graphs is to iterate over the edges in minibatches, since computing the probability of all edges is usually impossible. For each minibatch of edges, you compute the output representation of their incident nodes using neighbor sampling and GNN, in a similar fashion introduced in the node classification tutorial.

To perform link prediction, you need to specify a negative sampler. DGL provides builtin negative samplers such as `dgl.graphbolt.UniformNegativeSampler`. Here this tutorial uniformly draws 5 negative examples per positive example.

Except for the negative sampler, the rest of the code is identical to the node classification tutorial.



In [4]:
from functools import partial
datapipe = gb.ItemSampler(train_set, batch_size=256, shuffle=True)
datapipe = datapipe.sample_uniform_negative(graph, 5)
datapipe = datapipe.sample_neighbor(graph, [5, 5, 5])
datapipe = datapipe.transform(partial(gb.exclude_seed_edges, include_reverse_edges=True))
datapipe = datapipe.fetch_feature(feature, node_feature_keys=["feat"])
datapipe = datapipe.copy_to(device)
train_dataloader = gb.DataLoader(datapipe)

You can peek one minibatch from train_dataloader and see what it will give you.



In [5]:
data = next(iter(train_dataloader))
print(f"MiniBatch: {data}")

RuntimeError: sample_neighbors() Expected a value of type 'List[int]' for argument '_3' but instead found type 'bool'.
Position: 3
Value: False
Declaration: sample_neighbors(__torch__.torch.classes.graphbolt.FusedCSCSamplingGraph _0, Tensor? _1, int[]? _2, int[] _3, bool _4, bool _5, bool _6, Tensor? _7, Tensor? _8, float _9) -> __torch__.torch.classes.graphbolt.FusedSampledSubgraph _0
Cast error details: Unable to cast Python instance of type <class 'bool'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)
This exception is thrown by __iter__ of NeighborSampler(datapipe=UniformNegativeSampler, deduplicate=True, fanouts=[tensor([5]), tensor([5]), tensor([5])], graph=FusedCSCSamplingGraph(csc_indptr=tensor([    0,     3,     6,  ..., 10548, 10552, 10556]),
                      indices=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
                      num_nodes=2708, num_edges=10556, node_attributes={}, edge_attributes={}), prob_name=

## Defining Model for Node Representation
Let’s consider training a 2-layer GraphSAGE with neighbor sampling. The model can be written as follows:


In [6]:
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F


class SAGE(nn.Module):
    def __init__(self, in_size, hidden_size):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(dglnn.SAGEConv(in_size, hidden_size, "mean"))
        self.layers.append(dglnn.SAGEConv(hidden_size, hidden_size, "mean"))
        self.hidden_size = hidden_size
        self.predictor = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )

    def forward(self, blocks, x):
        hidden_x = x
        for layer_idx, (layer, block) in enumerate(zip(self.layers, blocks)):
            hidden_x = layer(block, hidden_x)
            is_last_layer = layer_idx == len(self.layers) - 1
            if not is_last_layer:
                hidden_x = F.relu(hidden_x)
        return hidden_x

## Defining Traing Loop
The following initializes the model and defines the optimizer.



In [7]:
in_size = feature.size("node", None, "feat")[0]
model = SAGE(in_size, 128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

The following is the training loop for link prediction and evaluation.



In [8]:
import tqdm
for epoch in range(3):
    model.train()
    total_loss = 0
    for step, data in tqdm.tqdm(enumerate(train_dataloader)):
        # Get node pairs with labels for loss calculation.
        compacted_pairs, labels = data.node_pairs_with_labels
        node_feature = data.node_features["feat"]
        # Convert sampled subgraphs to DGL blocks.
        blocks = data.blocks

        # Get the embeddings of the input nodes.
        y = model(blocks, node_feature)
        logits = model.predictor(
            y[compacted_pairs[0]] * y[compacted_pairs[1]]
        ).squeeze()

        # Compute loss.
        loss = F.binary_cross_entropy_with_logits(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch:03d} | Loss {total_loss / (step + 1):.3f}")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

RuntimeError: sample_neighbors() Expected a value of type 'List[int]' for argument '_3' but instead found type 'bool'.
Position: 3
Value: False
Declaration: sample_neighbors(__torch__.torch.classes.graphbolt.FusedCSCSamplingGraph _0, Tensor? _1, int[]? _2, int[] _3, bool _4, bool _5, bool _6, Tensor? _7, Tensor? _8, float _9) -> __torch__.torch.classes.graphbolt.FusedSampledSubgraph _0
Cast error details: Unable to cast Python instance of type <class 'bool'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)
This exception is thrown by __iter__ of NeighborSampler(datapipe=UniformNegativeSampler, deduplicate=True, fanouts=[tensor([5]), tensor([5]), tensor([5])], graph=FusedCSCSamplingGraph(csc_indptr=tensor([    0,     3,     6,  ..., 10548, 10552, 10556]),
                      indices=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
                      num_nodes=2708, num_edges=10556, node_attributes={}, edge_attributes={}), prob_name=

## Evaluating Performance with Link Prediction


In [9]:
model.eval()

datapipe = gb.ItemSampler(test_set, batch_size=256, shuffle=False)
# Since we need to use all neghborhoods for evaluation, we set the fanout
# to -1.
datapipe = datapipe.sample_neighbor(graph, [-1, -1])
datapipe = datapipe.fetch_feature(feature, node_feature_keys=["feat"])
datapipe = datapipe.copy_to(device)
eval_dataloader = gb.DataLoader(datapipe, num_workers=0)

logits = []
labels = []
for step, data in tqdm.tqdm(enumerate(eval_dataloader)):
    # Get node pairs with labels for loss calculation.
    compacted_pairs, label = data.node_pairs_with_labels

    # The features of sampled nodes.
    x = data.node_features["feat"]

    # Forward.
    y = model(data.blocks, x)
    logit = (
        model.predictor(y[compacted_pairs[0]] * y[compacted_pairs[1]])
        .squeeze()
        .detach()
    )

    logits.append(logit)
    labels.append(label)

logits = torch.cat(logits, dim=0)
labels = torch.cat(labels, dim=0)


# Compute the AUROC score.
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(labels.cpu(), logits.cpu())
print("Link Prediction AUC:", auc)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

RuntimeError: sample_neighbors() Expected a value of type 'List[int]' for argument '_3' but instead found type 'bool'.
Position: 3
Value: False
Declaration: sample_neighbors(__torch__.torch.classes.graphbolt.FusedCSCSamplingGraph _0, Tensor? _1, int[]? _2, int[] _3, bool _4, bool _5, bool _6, Tensor? _7, Tensor? _8, float _9) -> __torch__.torch.classes.graphbolt.FusedSampledSubgraph _0
Cast error details: Unable to cast Python instance of type <class 'bool'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)
This exception is thrown by __iter__ of NeighborSampler(datapipe=ShardingFilterIterDataPipe, deduplicate=True, fanouts=[tensor([-1]), tensor([-1])], graph=FusedCSCSamplingGraph(csc_indptr=tensor([    0,     3,     6,  ..., 10548, 10552, 10556]),
                      indices=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
                      num_nodes=2708, num_edges=10556, node_attributes={}, edge_attributes={}), prob_name=None, r

## Conclusion
In this tutorial, you have learned how to train a multi-layer GraphSAGE for link prediction with neighbor sampling.